### generate predictions using the percentage change between t-2 and t-1 and apply to t
then measure the results with r2 metric against the real data


In [23]:
import pandas as pd
import numpy as np
import bamboolib
from sklearn import preprocessing

In [24]:
data_2006_x = pd.read_csv('x_df_2006_data.csv')
data_2006_y = pd.read_csv('y_2006.csv')
data_2011_x = pd.read_csv('x_df_2011_data.csv')
data_2011_y = pd.read_csv('y_2011.csv')
data_2016_y = pd.read_csv('y_2016.csv')

In [25]:
print(data_2006_y.shape)
print(data_2011_y.shape)
print(data_2016_y.shape)

(572, 267)
(577, 267)
(685, 267)


In [26]:
#CONCATENATE THEM TOGETHER
result1 = pd.concat([data_2006_y, data_2011_y], ignore_index=True, sort=False)
y_stack = pd.concat([result1, data_2016_y], ignore_index=True, sort=False)
#drop the columns with Nan values
y_stack = y_stack.dropna(how={'any'},thresh = 10, axis=1)
y_stack

,Unnamed: 0,lga,Adelie Land (France),Afghanistan,Albania,Algeria,"Americas, nfd",Andorra,Angola,Anguilla,...,Vanuatu,"Virgin Islands, British","Virgin Islands, United States",Wales,Wallis and Futuna,"Western Europe, nfd",Western Sahara,Yemen,Zambia,Zimbabwe
0,0,Albury (C),0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,43.0,0.0,0.0,0.0,0.0,18.0,30.0
1,1,Armidale Regional (A),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.0,0.0,0.0,0.0,3.0,3.0,24.0
2,2,Ballina (A),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,53.0,0.0,0.0,0.0,0.0,0.0,33.0
3,3,Balranald (A),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Bathurst Regional (A),0.0,10.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,1829,No Usual Address,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1830,1830,Unincorporated ACT,0.0,233.0,12.0,13.0,0.0,0.0,15.0,0.0,...,37.0,0.0,0.0,424.0,0.0,0.0,0.0,14.0,90.0,327.0
1831,1831,No Usual Address,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1832,1832,Unincorporated OT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# splitting dataframe by row index
d2006_y = y_stack.iloc[:572,:]
d2011_y = y_stack.iloc[572:1149,:]
d2016_y = y_stack.iloc[1149:1836,:]

In [28]:
print(d2006_y.shape)
print(d2011_y.shape)
print(d2016_y.shape)

(572, 267)
(577, 267)
(685, 267)


In [29]:
#clean the LGAs so they are consistent accross the three years

y_2006_and_y_2011 = pd.merge(d2006_y, d2011_y, how='inner', on=['lga'])
y_2006_and_y_2011_y_2016 = pd.merge(y_2006_and_y_2011, d2016_y, how='inner', on=['lga'])


In [30]:
y_2006_and_y_2011_y_2016

,Unnamed: 0_x,lga,Adelie Land (France)_x,Afghanistan_x,Albania_x,Algeria_x,"Americas, nfd_x",Andorra_x,Angola_x,Anguilla_x,...,Vanuatu,"Virgin Islands, British","Virgin Islands, United States",Wales,Wallis and Futuna,"Western Europe, nfd",Western Sahara,Yemen,Zambia,Zimbabwe
0,0,Albury (C),0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,5.0,24.0
1,2,Ballina (A),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,52.0,0.0,0.0,0.0,0.0,8.0,17.0
2,3,Balranald (A),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
3,4,Bathurst Regional (A),0.0,10.0,0.0,5.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,3.0,6.0
4,5,Bega Valley (A),0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,542,Darwin (C),0.0,41.0,10.0,8.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,74.0,0.0,0.0,0.0,0.0,10.0,142.0
461,544,Katherine (T),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,18.0
462,547,Palmerston (C),0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,3.0,12.0
463,554,Unincorporated NT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,3.0,14.0


In [31]:
#and re-split again now that they are all LGA consistent
data_2006_y = y_2006_and_y_2011_y_2016.iloc[:, :268]
data_2011_y = y_2006_and_y_2011_y_2016.iloc[:, 267:534]
data_2016_y = y_2006_and_y_2011_y_2016.iloc[:, 533:800]



In [32]:

data_2006_y = data_2006_y.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'])
data_2011_y = data_2011_y.drop(columns=['Unnamed: 0_y','Unnamed: 0'])
data_2016_y = data_2016_y.drop(columns=['Unnamed: 0'])

In [33]:
lgas = data_2006_y['lga']
data_2006_y = data_2006_y.drop(columns=['lga'])
print(data_2006_y.shape)
print(data_2011_y.shape)
print(data_2016_y.shape)

(465, 265)
(465, 265)
(465, 265)


In [34]:
data_2006_y.to_numpy()
data_2011_y.to_numpy()
data_2016_y.to_numpy()

data_2006_y = data_2006_y.astype(np.float)
data_2011_y = data_2011_y.astype(np.float)
data_2016_y = data_2016_y.astype(np.float)

In [35]:
np.count_nonzero(np.isnan(data_2016_y))

0

In [36]:
#data_2011_nozeros = np.where(data_2011_y==0,data_2011_y, 1)
#data_2011_nozeros
data_2011_y[data_2011_y==0] = 1
data_2006_y[data_2006_y==0] = 1

In [37]:
sequence_rate = np.divide(data_2011_y , data_2006_y, where=data_2006_y!=0)

predictions = np.multiply(data_2011_y , sequence_rate)

In [38]:
predictions

,Adelie Land (France)_y,Afghanistan_y,Albania_y,Algeria_y,"Americas, nfd_y",Andorra_y,Angola_y,Anguilla_y,"Antarctica, nfd_y",Antigua and Barbuda_y,...,Vanuatu_y,"Virgin Islands, British_y","Virgin Islands, United States_y",Wales_y,Wallis and Futuna_y,"Western Europe, nfd_y",Western Sahara_y,Yemen_y,Zambia_y,Zimbabwe_y
0,1.0,0.111111,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,...,1.000000,1.0,1.0,45.023256,1.0,1.0,1.0,1.000,5.555556,30.000000
1,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,...,1.000000,1.0,1.0,74.886792,1.0,1.0,1.0,1.000,16.000000,17.454545
2,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,...,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000,1.000000,9.000000
3,1.0,0.900000,1.000000,9.800000,1.0,1.0,1.000000,1.0,1.0,1.0,...,36.000000,1.0,1.0,54.450000,1.0,1.0,1.0,1.000,9.000000,13.136364
4,1.0,1.000000,1.000000,0.333333,1.0,1.0,1.000000,1.0,1.0,1.0,...,9.000000,1.0,1.0,32.236842,1.0,1.0,1.0,1.000,5.333333,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,1.0,738.439024,0.100000,1.125000,1.0,1.0,9.000000,1.0,1.0,1.0,...,40.000000,1.0,1.0,70.583333,1.0,1.0,1.0,16.000,25.484848,274.716738
461,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,...,1.000000,1.0,1.0,1.285714,1.0,1.0,1.0,1.000,9.000000,57.760000
462,1.0,0.071429,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,...,1.000000,1.0,1.0,27.272727,1.0,1.0,1.0,1.000,0.529412,8.126866
463,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,...,0.250000,1.0,1.0,2.250000,1.0,1.0,1.0,1.000,0.166667,25.941176


In [39]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

baseline_score = r2_score(data_2016_y,predictions,multioutput='uniform_average')


#scoring metrics test data
score = r2_score(data_2016_y,predictions,multioutput='uniform_average')
mse= mean_squared_error(data_2016_y,predictions,multioutput='uniform_average')
mae = mean_absolute_error(data_2016_y,predictions,multioutput='uniform_average')


print("TEST ERROR")
print("rf_r2 ",score,"rf_mse ",mse,"rf_mae ",mae)

TEST ERROR
rf_r2  -378.0817697928761 rf_mse  18628964.918105207 rf_mae  35.59401776273475
